In [1]:
from itertools import zip_longest
import polars as pl
from tqdm import tqdm

In [2]:
def load2df(file):
    with open(file) as fin:
        lines = fin.readlines()
        
    records = []
    tmp_record = {}
    is_first = True
    for line in tqdm(lines):
        line = line.replace("\n", "")

        first = ("|" in line)
        second = line.startswith("WEDNESDAY")
        third = line.startswith("THURSDAY")    

        # print(first, second, third)
        if first and (second or third):
            DAY = line.split("|")[0]
            TIME = line.split("|")[1]
            continue

        added_flag = False


        for key in ["Speaker:","Track:", "Tracks:", "Format:", "Location:", "Contributor:", "Panelist:", "Keynote:", "Moderator:", "Experience:"]:
            if line.startswith(key):
                assert key not in tmp_record, f"{line}, {key} {tmp_record}"
                added_key = key[:-1]

                if "Contributor:" == key:
                    added_key = "Speaker"
                if "Tracks:" == key:
                    added_key = "Track"
                if "Moderator" == key:
                    added_key = "Speaker"
                if "Keynote:" == key:
                    added_key = "Main"
                


                tmp_record[added_key] = ":".join(line.split(":")[1:])
                added_flag = True

        if not added_flag:
            assert "Main" not in tmp_record, f"{line} main {tmp_record}"
            tmp_record["Main"] = line

        if line.startswith("Location:"):
            tmp_record["Date"]= DAY
            tmp_record["Time"]= TIME
            records.append(tmp_record)

            tmp_record = {}
    return pl.DataFrame(records)

def load_session(file):
    with open(file, "r") as fin:
        return [a.strip() for a in fin.readlines()]    

In [3]:
df_2023 = load2df('2023_sessions.txt')
tracks_2023 = load_session('2023_tracks.txt')

100%|██████████| 590/590 [00:00<00:00, 243088.35it/s]


In [4]:
dict_2023={}
for target_label in tracks_2023:
    dict_2023[target_label] = df_2023.filter(pl.col("Track").str.contains(target_label)).shape[0]

sorted_keys_2023 = sorted(dict_2023, key=dict_2023.get, reverse=True)

In [5]:
for a in sorted_keys_2023:
    print(a, dict_2023[a])

Defense 17
AI, ML, & Data Science 15
Exploit Development 14
Application Security: Offense 13
Enterprise Security 13
Hardware / Embedded 12
Application Security: Defense 11
Cloud Security 11
Data Forensics & Incident Response 11
Network Security 10
Platform Security 10
Cyber-Physical Systems & IoT 9
Mobile 8
Reverse Engineering 8
Cryptography 7
Malware 7
Policy 6
Human Factors 5
Cyber Insurance 4
Entrepreneur 4
Privacy 4
Community & Career 3
Keynote 3


AI, ML, & Data Science: 5 -> 15
    - 
    - 

Application Security: 17 → 24
Enterprise Sesecurity: 17 → 24
Data Forensics & Incident Response: 4 → 11 

Cyber insurance: 新規軸
    -　



In [6]:
df_2022 = load2df('2022_sessions.txt')
tracks_2022 = load_session('2022_tracks.txt')

100%|██████████| 659/659 [00:00<00:00, 151446.30it/s]


In [7]:
dict_2022={}
for target_label in tracks_2022:
    dict_2022[target_label] = df_2022.filter(pl.col("Track").str.contains(target_label)).shape[0]

sorted_keys_2022 = sorted(dict_2022, key=dict_2022.get, reverse=True)

In [8]:
for a in sorted_keys_2022:
    print(a, dict_2022[a])

Cloud & Platform Security 21
Application Security 17
Defense 15
Exploit Development 15
Hardware / Embedded 14
Reverse Engineering 13
Network Security 12
Lessons Learned 9
Malware 9
Mobile 9
Enterprise Security 7
Cyber-Physical Systems 6
Human Factors 6
Policy 6
AI, ML, & Data Science 5
Cryptography 5
Privacy 5
Community & Career 4
Data Forensics & Incident Response 4
Keynote 3


In [14]:
a = set(sorted_keys_2023)
b = set(sorted_keys_2022)

In [17]:
a - b

{'Application Security: Defense',
 'Application Security: Offense',
 'Cloud Security',
 'Cyber Insurance',
 'Cyber-Physical Systems & IoT',
 'Entrepreneur',
 'Platform Security'}

In [16]:
b

{'AI, ML, & Data Science',
 'Application Security',
 'Cloud & Platform Security',
 'Community & Career',
 'Cryptography',
 'Cyber-Physical Systems',
 'Data Forensics & Incident Response',
 'Defense',
 'Enterprise Security',
 'Exploit Development',
 'Hardware / Embedded',
 'Human Factors',
 'Keynote',
 'Lessons Learned',
 'Malware',
 'Mobile',
 'Network Security',
 'Policy',
 'Privacy',
 'Reverse Engineering'}